In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
import datetime
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("..") # Adds higher directory to python modules path
import geodict
tofullcensus = geodict.tofullcensus
geotogeoid = geodict.geotogeoid
import sqlite3 as sq
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
def percent(x, y):
    return (x/y)*100

Quarterly Employment During the Heart of COVID 19 Pandemic

The Woods & Poole data that is used more at the county level is only used at the place level for total employment projections. This is for consistency. This is a land use model ultimately, so JobsEQ employment is selected to model place level employment for place of work. This data is derived from the QCEW and other inputs so is selected as a better option - both are modeled so we use the one modeled directly from UI claims rather than land use at this granular of a geography.

To retrieve data:
+ Log into JobsEQ and navigate to the "Data Explorer" function  
+ go to "Draft Mode"
+ Select the "Industry" dataset, quarterly  
+ Select the following dimensions: Region, Industry, Year: fill regions with all geographies including US, State, GNRC (13), MPO (7), GNRC Region (14) and sub all counties, all places  
+ Fill the year/quarter with 2019-current (in the future change this to relevant time period if need be)  
+ select total and high level industries as listed below:  

Industries to Select:  
(NAICS Codes)  
+ Total - All Industries  
+ Construction (23)  
+ Manufacturing (31)  
+ Information (51)  
+ Public Administration (92)  
(Custom Created Groups *begin to type them as they appear here and they will appear in the search bar, if not then remake the custom group as described below*)  
+ Other/Unclassified (81: Other, Except Public Administration, 99: Unclassified)  
+ Leisure & Hospitality (71: Arts, Entertainment, & Recreation, 72: Accommodation & Food Services)   
+ Education & Health Services (61: Education, 62: Healthcare & Social Assistance)   
+ Professional & Business Services (54: Professional, Scientific, & Technical, 55: Management of Companies & Enterprises, 56: Administrative & Support & Waste Management & Remediation)   
+ Financial Activities (52: Finance & Insurance, 53: Real Estate & Rental & Leasing)   
+ Trade, Transportation & Utilities (42: Wholesale Trade, 44-45: Retail Trade, 48-49: Transportation & Warehousing, 22: Utilities)  
+ Natural Resources & Mining (11: Agricultural Activities, Forestry, Fishing, & Hunting, 21: Mining, Quarrying, Oil & Gas)  

+ Select the following values: "Employment (4Q Moving Average)"

Once it is completed, open and delete the top row that says "Industry Data", and adjust so that the column headers reflect the industry, as it is all the same datapoint. Delete the FIPS column. Rename the Region column to NAME and the quarter column to Time, delete the data notes at the bottom. Save as:   "JobsEQ_IndustryEmploymentCOVIDQuarterly_AllGeos.csv"

In [8]:
#import data and examine
data = pd.read_csv('../../Data Downloads/JobsEQ_IndustryEmploymentCOVIDQuarterly_AllGeos.csv')
data.head(2)

,NAME,Time,Total - All Industries,Construction (23),Manufacturing (31),Information (51),Public Administration (92),Other/Unclassified,Leisure & Hospitality,Education & Health Services,Professional & Business Services,Financial Activities,"Trade, Transportation, & Utilities",Natural Resources & Mining
0,USA,2019Q1,158031299.4,9094807.625,13015409.16,3081771.166,7426728.516,7248880.031,17441949.38,35570293.91,23193839.00,8966304.61,30156671.43,2834644.558
1,USA,2019Q2,158451024.1,9153189.405,13056876.64,3089322.210,7440921.997,7239161.091,17510564.61,35698475.54,23276916.36,8989721.82,30168449.41,2827424.982


In [9]:
#quickly make sure the geographies are the full census geographies
data = data.set_index('NAME').transpose()
data = data.rename(columns = tofullcensus)
data = data.transpose().reset_index(drop = False)

In [10]:
#make a list of the times to loop through unique instances for aggregating geographies later
times = list(data['Time'].unique())

In [11]:
#set the index to name and time, creating unique instances for each industry group, transpose, and make sure the columns are float datatypes
data = data.set_index(['NAME', 'Time']).transpose()
cols = data.columns
data[cols] = data[cols].astype(float)

In [12]:
data.head(2)

NAME                   United States                                                                                                    \
Time                          2019Q1        2019Q2        2019Q3        2019Q4        2020Q1        2020Q2        2020Q3        2020Q4   
Total - All Industries  1.580313e+08  1.584510e+08  1.589271e+08  1.593542e+08  1.596925e+08  1.551343e+08  1.522915e+08  1.500387e+08   
Construction (23)       9.094808e+06  9.153189e+06  9.218993e+06  9.269804e+06  9.313434e+06  9.135583e+06  9.037460e+06  8.965810e+06   

NAME                                                                                                                         Tennessee               \
Time                          2021Q1        2021Q2        2021Q3        2021Q4        2022Q1        2022Q2        2022Q3        2019Q1       2019Q2   
Total - All Industries  1.480799e+08  1.514103e+08  1.534816e+08  1.553511e+08  1.572637e+08  1.589226e+08  1.602289e+08  3.262497e+06  3275823.640   
Construction (23)       8.915467e+06  9.107383e+06  9.218492e+06  9.314619e+06  9.399146e+06  9.468230e+06  9.518560e+06  1.716012e+05   173199.552   

NAME                                                                                                                                    \
Time                          2019Q3        2019Q4        2020Q1        2020Q2        2020Q3        2020Q4        2021Q1        2021Q2   
Total - All Industries  3.290548e+06  3.301694e+06  3.311384e+06  3.238748e+06  3.199422e+06  3.177383e+06  3.156862e+06  3.220612e+06   
Construction (23)       1.745552e+05  1.759386e+05  1.769721e+05  1.754027e+05  1.746940e+05  1.744530e+05  1.749228e+05  1.781304e+05   

NAME                                                                                                 GNRC                                            \
Time                          2021Q3       2021Q4        2022Q1        2022Q2        2022Q3        2019Q1        2019Q2        2019Q3        2019Q4   
Total - All Industries  3.262401e+06  3290422.782  3.326595e+06  3.365929e+06  3.393472e+06  1.068753e+06  1.077372e+06  1.086029e+06  1.094040e+06   
Construction (23)       1.813275e+05   184124.694  1.867191e+05  1.893454e+05  1.912969e+05  5.834205e+04  5.917056e+04  5.992698e+04  6.081850e+04   

NAME                                                                                                                                    \
Time                          2020Q1        2020Q2        2020Q3        2020Q4        2021Q1        2021Q2        2021Q3        2021Q4   
Total - All Industries  1.099525e+06  1.071923e+06  1.056044e+06  1.046731e+06  1.038517e+06  1.064138e+06  1.082450e+06  1.095707e+06   
Construction (23)       6.135000e+04  6.094352e+04  6.074474e+04  6.067852e+04  6.094481e+04  6.206863e+04  6.334915e+04  6.462682e+04   

NAME                                                                       MPO                                                          \
Time                          2022Q1        2022Q2        2022Q3        2019Q1        2019Q2        2019Q3        2019Q4        2020Q1   
Total - All Industries  1.111473e+06  1.127325e+06  1.137806e+06  1.004341e+06  1.013088e+06  1.021745e+06  1.029205e+06  1.034503e+06   
Construction (23)       6.574951e+04  6.695720e+04  6.769146e+04  5.302325e+04  5.381781e+04  5.455947e+04  5.545667e+04  5.599609e+04   

NAME                                                                                                                                    \
Time                          2020Q2        2020Q3        2020Q4        2021Q1        2021Q2        2021Q3        2021Q4        2022Q1   
Total - All Industries  1.006952e+06  991215.45030  982071.19150  973699.50090  998596.06270  1.015961e+06  1.028407e+06  1.043633e+06   
Construction (23)       5.570255e+04   55589.95145   55539.77437   55792.74973   56837.66216  5.802574e+04  5.923482e+04  6.027076e+04   

NAME                     

In [13]:
#loop through the times we listed earlier to aggregate the incorporated and unincorporated areas
for time in times:
    RuthInc = [data[('Eagleville city, Tennessee', '{}'.format(time))],data[('La Vergne city, Tennessee', '{}'.format(time))],
               data[('Murfreesboro city, Tennessee', '{}'.format(time))],data[('Smyrna town, Tennessee', '{}'.format(time))]]
    data[('Rutherford Incorporated', '{}'.format(time))] = sum(RuthInc)
    data[('Rutherford Unincorporated', '{}'.format(time))] = data[('Rutherford County, Tennessee', '{}'.format(time))] - data[('Rutherford Incorporated', '{}'.format(time))]
    WilsonInc = [data[('Lebanon city, Tennessee', '{}'.format(time))],data[('Mount Juliet city, Tennessee', '{}'.format(time))],
                 data[('Watertown city, Tennessee', '{}'.format(time))]]
    data[('Wilson Incorporated', '{}'.format(time))] = sum(WilsonInc)
    data[('Wilson Unincorporated', '{}'.format(time))] = data[('Wilson County, Tennessee', '{}'.format(time))] - data[('Wilson Incorporated', '{}'.format(time))]
    CheathInc = [data[('Ashland City town, Tennessee', '{}'.format(time))],data[('Kingston Springs town, Tennessee', '{}'.format(time))],
                 data[('Pegram town, Tennessee', '{}'.format(time))],data[('Pleasant View city, Tennessee', '{}'.format(time))]]
    data[('Cheatham Incorporated', '{}'.format(time))] = sum(CheathInc)
    data[('Cheatham Unincorporated', '{}'.format(time))] = data[('Cheatham County, Tennessee', '{}'.format(time))] - data[('Cheatham Incorporated', '{}'.format(time))]
    DicksInc = [data[('Burns town, Tennessee', '{}'.format(time))],data[('Charlotte town, Tennessee', '{}'.format(time))],
                data[('Dickson city, Tennessee', '{}'.format(time))],data[('Slayden town, Tennessee', '{}'.format(time))],
                data[('Vanleer town, Tennessee', '{}'.format(time))],data[('White Bluff town, Tennessee', '{}'.format(time))]]
    data[('Dickson Incorporated', '{}'.format(time))] = sum(DicksInc)
    data[('Dickson Unincorporated', '{}'.format(time))] = data[('Dickson County, Tennessee', '{}'.format(time))] - data[('Dickson Incorporated', '{}'.format(time))]
    HumphInc = [data[('McEwen city, Tennessee', '{}'.format(time))],data[('New Johnsonville city, Tennessee', '{}'.format(time))],
                data[('Waverly city, Tennessee', '{}'.format(time))]]
    data[('Humphreys Incorporated', '{}'.format(time))] = sum(HumphInc)
    data[('Humphreys Unincorporated', '{}'.format(time))] = data[('Humphreys County, Tennessee', '{}'.format(time))] - data[('Humphreys Incorporated', '{}'.format(time))]
    data[('Montgomery Incorporated', '{}'.format(time))] = data[('Clarksville city, Tennessee', '{}'.format(time))]
    data[('Montgomery Unincorporated', '{}'.format(time))] = data[('Montgomery County, Tennessee', '{}'.format(time))] - data[('Montgomery Incorporated', '{}'.format(time))]

In [14]:
#transpose and check
data = data.transpose()
data.head(2)

Total - All Industries  Construction (23)  Manufacturing (31)  Information (51)  Public Administration (92)  \
NAME          Time                                                                                                                  
United States 2019Q1             158031299.4        9094807.625         13015409.16       3081771.166                 7426728.516   
              2019Q2             158451024.1        9153189.405         13056876.64       3089322.210                 7440921.997   

                      Other/Unclassified  Leisure & Hospitality  Education & Health Services  Professional & Business Services  \
NAME          Time                                                                                                               
United States 2019Q1         7248880.031            17441949.38                  35570293.91                       23193839.00   
              2019Q2         7239161.091            17510564.61                  35698475.54                       23276916.36   

                      Financial Activities  Trade, Transportation, & Utilities  Natural Resources & Mining  
NAME          Time                                                                                          
United States 2019Q1            8966304.61                         30156671.43                 2834644.558  
              2019Q2            8989721.82                         30168449.41                 2827424.982

In [15]:
#sum these, note that this categorization for service producing doesn't contain trade, transportation & utilities as that's broken out separately
thelist = [data['Natural Resources & Mining'], data['Construction (23)'], data['Manufacturing (31)']]
data['Goods Producing'] = sum(thelist)
thelist = [data['Information (51)'], data['Financial Activities'], data['Professional & Business Services'], 
           data['Education & Health Services'], data['Leisure & Hospitality'], data['Public Administration (92)'], data['Other/Unclassified']]
data['Service Producing'] = sum(thelist)

In [16]:
#resum so that percentages work - data caveat that they would not sum perfectly due to rounding
thelist = [data['Construction (23)'], data['Manufacturing (31)'], 
           data['Information (51)'], data['Public Administration (92)'], 
           data['Other/Unclassified'], data['Leisure & Hospitality'],
           data['Education & Health Services'], data['Professional & Business Services'], 
           data['Financial Activities'], data['Trade, Transportation, & Utilities'], data['Natural Resources & Mining']]
data['Total - All Industries'] = sum(thelist)

In [17]:
#rename the industry groups
data = data.rename(columns = {'Total - All Industries': 'Total', 'Construction (23)': 'Construction', 'Manufacturing (31)': 'Manufacturing', 
                              'Information (51)': 'Information', 'Public Administration (92)': 'Public Administration'})

In [18]:
#set the index to year and name so that we can add a prefix to all of the other columns, before resetting the index without dropping
data.reset_index(drop = False, inplace = True)
data = data.set_index(['NAME', 'Time']).add_prefix('Ind:').reset_index(drop = False)

In [21]:
#name cols as all of the columns and then go from wide to long format with .melt(), call this thing because it gets buggy if you call if data 
cols = data.columns
thing = data.melt(var_name = 'Industry', id_vars = ['NAME', 'Time'], value_vars = cols, value_name = 'Employment')

In [22]:
thing.head(2)

,NAME,Time,Industry,Employment
0,United States,2019Q1,Ind:Total,1.580313e+08
1,United States,2019Q2,Ind:Total,1.584510e+08


In [23]:
#create a separate dataframe where we will calculate percent of the total for each occupation group
percents = data

In [24]:
#calculate percent of total
percents['Ind:Construction'] = percent(percents['Ind:Construction'], percents['Ind:Total'])
percents['Ind:Manufacturing'] = percent(percents['Ind:Manufacturing'], percents['Ind:Total'])
percents['Ind:Information'] = percent(percents['Ind:Information'], percents['Ind:Total'])
percents['Ind:Public Administration'] = percent(percents['Ind:Public Administration'], percents['Ind:Total'])
percents['Ind:Other/Unclassified'] = percent(percents['Ind:Other/Unclassified'], percents['Ind:Total'])
percents['Ind:Leisure & Hospitality'] = percent(percents['Ind:Leisure & Hospitality'], percents['Ind:Total'])
percents['Ind:Education & Health Services'] = percent(percents['Ind:Education & Health Services'], percents['Ind:Total'])
percents['Ind:Professional & Business Services'] = percent(percents['Ind:Professional & Business Services'], percents['Ind:Total'])
percents['Ind:Financial Activities'] = percent(percents['Ind:Financial Activities'], percents['Ind:Total'])
percents['Ind:Trade, Transportation, & Utilities'] = percent(percents['Ind:Trade, Transportation, & Utilities'], percents['Ind:Total'])
percents['Ind:Natural Resources & Mining'] = percent(percents['Ind:Natural Resources & Mining'], percents['Ind:Total'])
percents['Ind:Goods Producing'] = percent(percents['Ind:Goods Producing'], percents['Ind:Total'])
percents['Ind:Service Producing'] = percent(percents['Ind:Service Producing'], percents['Ind:Total'])

In [25]:
#similarly melt this dataframe
cols = percents.columns
percents = percents.reset_index(drop = False).melt(var_name = 'Industry', id_vars = ['NAME', 'Time'], value_vars = cols, value_name = 'Percent of Employment')
percents.head(2)

,NAME,Time,Industry,Percent of Employment
0,United States,2019Q1,Ind:Total,1.580313e+08
1,United States,2019Q2,Ind:Total,1.584510e+08


In [26]:
#we never renamed the total column when we duplicated the dataframe for percents, we want this to be null so just transpose, drop it, and transpose back
percents = percents.set_index('Industry').transpose()
percents = percents.drop(columns = 'Ind:Total')
percents = percents.transpose().reset_index(drop = False)

In [27]:
#merge total employment and percent of total employment
data = thing.merge(percents, on = ['Industry', 'Time', 'NAME'], how = 'outer')
data.head(3)

,NAME,Time,Industry,Employment,Percent of Employment
0,United States,2019Q1,Ind:Total,1.580313e+08,NaN
1,United States,2019Q2,Ind:Total,1.584510e+08,NaN
2,United States,2019Q3,Ind:Total,1.589271e+08,NaN


In [28]:
#remove the CDPs from the geographies, then transpose and use dictionary from custom module to rename geos beforeresetting the index
data = data.loc[~data['NAME'].str.contains('CDP')]
data = data.set_index('NAME').transpose().reset_index(drop = False)

In [29]:
#set the index to "index" (the variable names) and transpose, then reset the index without dropping it
data = data.set_index('index').transpose().reset_index(drop = False)

In [30]:
data.head(2)

index,NAME,Time,Industry,Employment,Percent of Employment
0,United States,2019Q1,Ind:Total,158031299.386,NaN
1,United States,2019Q2,Ind:Total,158451024.065,NaN


In [31]:
data.tail(2)

index,NAME,Time,Industry,Employment,Percent of Employment
18058,Montgomery Incorporated,2022Q3,Ind:Service Producing,36118.437325,66.421407
18059,Montgomery Unincorporated,2022Q3,Ind:Service Producing,4195.078775,39.583149


In [32]:
#go long to wide
data = pd.pivot(data, index = ['NAME', 'Time'], columns = 'Industry', values = ['Employment', 'Percent of Employment'])

In [33]:
#we'll have to flatten the multicolumns and then rename them to match the header formatting of our other data
data.columns = data.columns.to_flat_index()

In [34]:
#rename
inddict = {('Employment', 'Ind:Education & Health Services'): 'Ind:Education & Health Services', 
           ('Employment', 'Ind:Construction'): 'Ind:Construction', 
           ('Employment', 'Ind:Financial Activities'): 'Ind:Financial Activities',
           ('Employment', 'Ind:Information'): 'Ind:Information', 
           ('Employment', 'Ind:Manufacturing'): 'Ind:Manufacturing', 
           ('Employment', 'Ind:Other/Unclassified'): 'Ind:Other/Unclassified', 
           ('Employment', 'Ind:Public Administration'): 'Ind:Public Administration', 
           ('Employment', 'Ind:Total'): 'Ind:Total', 
           ('Employment', 'Ind:Goods Producing'): 'Ind:Goods Producing', 
           ('Employment', 'Ind:Leisure & Hospitality'): 'Ind:Leisure & Hospitality', 
           ('Employment', 'Ind:Natural Resources & Mining'): 'Ind:Natural Resources & Mining', 
           ('Employment', 'Ind:Professional & Business Services'): 'Ind:Professional & Business Services', 
           ('Employment', 'Ind:Service Producing'): 'Ind:Service Producing', 
           ('Employment', 'Ind:Trade, Transportation, & Utilities'): 'Ind:Trade, Transportation, & Utilities',
           ('Percent of Employment', 'Ind:Education & Health Services'): 'Ind%:Education & Health Services',
           ('Percent of Employment', 'Ind:Construction'): 'Ind%:Construction', 
           ('Percent of Employment', 'Ind:Financial Activities'): 'Ind%:Financial Activities',
           ('Percent of Employment', 'Ind:Information'): 'Ind%:Information', 
           ('Percent of Employment', 'Ind:Manufacturing'): 'Ind%:Manufacturing', 
           ('Percent of Employment', 'Ind:Other/Unclassified'): 'Ind%:Other/Unclassified', 
           ('Percent of Employment', 'Ind:Public Administration'): 'Ind%:Public Administration', 
           ('Percent of Employment', 'Ind:Total'): 'Ind%:Total', 
           ('Percent of Employment', 'Ind:Goods Producing'): 'Ind%:Goods Producing', 
           ('Percent of Employment', 'Ind:Leisure & Hospitality'): 'Ind%:Leisure & Hospitality', 
           ('Percent of Employment', 'Ind:Natural Resources & Mining'): 'Ind%:Natural Resources & Mining', 
           ('Percent of Employment', 'Ind:Professional & Business Services'): 'Ind%:Professional & Business Services', 
           ('Percent of Employment', 'Ind:Service Producing'): 'Ind%:Service Producing', 
           ('Percent of Employment', 'Ind:Trade, Transportation, & Utilities'): 'Ind%:Trade, Transportation, & Utilities'
           }
data = data.rename(columns = inddict)
data = data.reset_index(drop = False)

In [36]:
data.head(2)

,NAME,Time,Ind:Construction,Ind:Education & Health Services,Ind:Financial Activities,Ind:Goods Producing,Ind:Information,Ind:Leisure & Hospitality,Ind:Manufacturing,Ind:Natural Resources & Mining,Ind:Other/Unclassified,Ind:Professional & Business Services,Ind:Public Administration,Ind:Service Producing,Ind:Total,"Ind:Trade, Transportation, & Utilities",Ind%:Construction,Ind%:Education & Health Services,Ind%:Financial Activities,Ind%:Goods Producing,Ind%:Information,Ind%:Leisure & Hospitality,Ind%:Manufacturing,Ind%:Natural Resources & Mining,Ind%:Other/Unclassified,Ind%:Professional & Business Services,Ind%:Public Administration,Ind%:Service Producing,Ind%:Total,"Ind%:Trade, Transportation, & Utilities"
0,"Adams city, Tennessee",2019Q1,1.305952,16.306345,3.022547,2.244575,0.035368,9.18487,0.000106,0.938517,2.794552,1.051732,3.650866,36.046279,60.957684,22.66683,2.142391,26.750269,4.958434,3.682185,0.058021,15.067617,0.000174,1.53962,4.584413,1.725348,5.98918,59.133282,NaN,37.184533
1,"Adams city, Tennessee",2019Q2,0.97377,16.21505,3.007493,1.897441,0.032259,8.781033,0.000102,0.923569,2.843634,1.138403,3.264177,35.282049,59.517389,22.337899,1.636111,27.244222,5.053134,3.188045,0.054201,14.753727,0.000171,1.551764,4.777821,1.912723,5.48441,59.280237,NaN,37.531718


Use datetime to convert the Time column to a quarterly period.

https://stackoverflow.com/questions/53898482/clean-way-to-convert-quarterly-periods-to-datetime-in-pandas

In [37]:
qs = data['Time'].str.replace(r'(Q\d) (\d+)', r'\2-\1')

C:\Users\jmccall\AppData\Local\Temp\ipykernel_1924\2473378646.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  qs = data['Time'].str.replace(r'(Q\d) (\d+)', r'\2-\1')


In [38]:
#create a time stamp
data['DateTime'] = pd.PeriodIndex(qs, freq = 'Q').to_timestamp()

In [39]:
#before adding other identifier columns convert numerical to float
cols = list(data.columns)
cols.remove('NAME')
cols.remove('Time')
cols.remove('DateTime')
data[cols] = data[cols].astype(float)

In [40]:
#check that this worked
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1290 entries, 0 to 1289
Data columns (total 31 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   NAME                                     1290 non-null   object        
 1   Time                                     1290 non-null   object        
 2   Ind:Construction                         1290 non-null   float64       
 3   Ind:Education & Health Services          1290 non-null   float64       
 4   Ind:Financial Activities                 1290 non-null   float64       
 5   Ind:Goods Producing                      1290 non-null   float64       
 6   Ind:Information                          1290 non-null   float64       
 7   Ind:Leisure & Hospitality                1290 non-null   float64       
 8   Ind:Manufacturing                        1290 non-null   float64       
 9   Ind:Natural Resources & Mining           

In [41]:
#insert the datetime after the geography name and drop the datetime column
data.insert(1, 'TimeStamp', data['DateTime'])
data = data.drop(columns = 'DateTime')

In [42]:
#map to geoid dictionary from module and add source
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'JobsEQ'

In [43]:
#final check
data.head()

,NAME,TimeStamp,Time,Ind:Construction,Ind:Education & Health Services,Ind:Financial Activities,Ind:Goods Producing,Ind:Information,Ind:Leisure & Hospitality,Ind:Manufacturing,Ind:Natural Resources & Mining,Ind:Other/Unclassified,Ind:Professional & Business Services,Ind:Public Administration,Ind:Service Producing,Ind:Total,"Ind:Trade, Transportation, & Utilities",Ind%:Construction,Ind%:Education & Health Services,Ind%:Financial Activities,Ind%:Goods Producing,Ind%:Information,Ind%:Leisure & Hospitality,Ind%:Manufacturing,Ind%:Natural Resources & Mining,Ind%:Other/Unclassified,Ind%:Professional & Business Services,Ind%:Public Administration,Ind%:Service Producing,Ind%:Total,"Ind%:Trade, Transportation, & Utilities",GEO_ID,Source
0,"Adams city, Tennessee",2019-01-01,2019Q1,1.305952,16.306345,3.022547,2.244575,0.035368,9.184870,0.000106,0.938517,2.794552,1.051732,3.650866,36.046279,60.957684,22.666830,2.142391,26.750269,4.958434,3.682185,0.058021,15.067617,0.000174,1.539620,4.584413,1.725348,5.989180,59.133282,NaN,37.184533,1600000US4700200,JobsEQ
1,"Adams city, Tennessee",2019-04-01,2019Q2,0.973770,16.215050,3.007493,1.897441,0.032259,8.781033,0.000102,0.923569,2.843634,1.138403,3.264177,35.282049,59.517389,22.337899,1.636111,27.244222,5.053134,3.188045,0.054201,14.753727,0.000171,1.551764,4.777821,1.912723,5.484410,59.280237,NaN,37.531718,1600000US4700200,JobsEQ
2,"Adams city, Tennessee",2019-07-01,2019Q3,0.645157,16.136415,2.970006,1.567421,0.032931,8.465746,0.000098,0.922167,2.914760,1.237681,2.874473,34.632012,57.964825,21.765392,1.113014,27.838289,5.123807,2.704090,0.056811,14.604972,0.000169,1.590907,5.028498,2.135227,4.958995,59.746599,NaN,37.549310,1600000US4700200,JobsEQ
3,"Adams city, Tennessee",2019-10-01,2019Q4,0.315193,16.983101,2.901212,1.225575,0.033157,8.138603,0.000094,0.910288,2.883152,1.335975,2.484836,34.760037,56.900578,20.914967,0.553936,29.846975,5.098738,2.153888,0.058272,14.303199,0.000165,1.599787,5.067000,2.347911,4.366979,61.089075,NaN,36.757037,1600000US4700200,JobsEQ
4,"Adams city, Tennessee",2020-01-01,2020Q1,0.315552,18.009454,2.928454,1.193624,0.036952,8.267085,0.000094,0.877978,3.012317,1.356741,2.464372,36.075376,57.990243,20.721243,0.544147,31.056008,5.049909,2.058319,0.063721,14.255993,0.000162,1.514010,5.194523,2.339602,4.249633,62.209389,NaN,35.732292,1600000US4700200,JobsEQ


In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1290 entries, 0 to 1289
Data columns (total 33 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   NAME                                     1290 non-null   object        
 1   TimeStamp                                1290 non-null   datetime64[ns]
 2   Time                                     1290 non-null   object        
 3   Ind:Construction                         1290 non-null   float64       
 4   Ind:Education & Health Services          1290 non-null   float64       
 5   Ind:Financial Activities                 1290 non-null   float64       
 6   Ind:Goods Producing                      1290 non-null   float64       
 7   Ind:Information                          1290 non-null   float64       
 8   Ind:Leisure & Hospitality                1290 non-null   float64       
 9   Ind:Manufacturing                        

In [45]:
#export to the SQLite database as only the JobsEQ quarterly data
conn = sq.connect('../../Outputs/Labor_Economy.db')
data.to_sql('JobsEQ_IndustryEmployment_Quarterly', conn, if_exists = 'replace', index = False)

1290